In [ ]:
! pip install -q datasets transformers seqeval
!pip install -q corus razdel

     |████████████████████████████████| 290 kB 13.3 MB/s 
     |████████████████████████████████| 3.1 MB 45.0 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 30.8 MB/s 
     |████████████████████████████████| 59 kB 4.8 MB/s 
     |████████████████████████████████| 243 kB 48.1 MB/s 
     |████████████████████████████████| 125 kB 48.4 MB/s 
     |████████████████████████████████| 895 kB 50.9 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 160 kB 52.0 MB/s 
     |████████████████████████████████| 192 kB 54.7 MB/s 
     |████████████████████████████████| 271 kB 47.9 MB/s 
     |████████████████████████████████| 83 kB 1.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric, Dataset, DatasetDict
from razdel import tokenize
import torch
from sklearn.model_selection import train_test_split
import os

from transformers import set_seed

In [ ]:
def seed_everything(seed: int):
    """Seeds and fixes every possible random state."""
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    set_seed(seed)

SEED = 3407
seed_everything(SEED)

In [ ]:
model_checkpoint = "cointegrated/rubert-tiny2"
batch_size = 4

In [ ]:
!wget -q -O train.csv https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/fmWGQJvwU5ejog
!wget -q -O train_labels.json https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/0nJ2QTRb9-U7tA
!wget -q https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

In [ ]:
from corus import load_rudrec
drugs = list(load_rudrec('rudrec_annotated.json'))
print(len(drugs))

4809


In [ ]:
train_df = pd.read_csv('train.csv')
train_labels = pd.read_json('train_labels.json').T

In [ ]:
labeled_by_hand_df = pd.read_json('project-1-at-2021-11-05-17-15-b0971356.json')
labeled_by_hand_df['id'] = labeled_by_hand_df['id'] - np.ones(len(labeled_by_hand_df['id']))
labeled_by_hand_df['id'] = labeled_by_hand_df['id'].astype(int)

In [ ]:
from razdel import tokenize

def extract_labels_drug(item):
    raw_toks = list(tokenize(item.text))
    words = [tok.text for tok in raw_toks]
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item.text)
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    for e in item.entities:
        if e.entity_type == 'ADR':
            e_words = sorted({idx for idx in char2word[e.start:e.end] if idx is not None})
            word_labels[e_words[0]] = 'B-' + e.entity_type
            for idx in e_words[1:]:
                word_labels[idx] = 'I-' + e.entity_type

    if "B-ADR" not in word_labels:
        return 0

    return {'tokens': words, 'tags': word_labels}

In [ ]:
def extract_labels(train_index):
    text = train_df.text[train_index]
    raw_toks = list(tokenize(text))
    words = [tok.text for tok in raw_toks]

    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(text)

    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    spans = train_labels.span[train_index]
    for e in spans:
        entity_type = "ADR"
        e_words = sorted({idx for idx in char2word[e[0]:e[1]] if idx is not None})
        word_labels[e_words[0]] = 'B-' + entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + entity_type

    return {'tokens': words, 'tags': word_labels}

In [ ]:
def extract_labels_by_hand(labeled_by_hand_index):
    text = labeled_by_hand_df['data'][labeled_by_hand_index]['text']


    raw_toks = list(tokenize(text))
    words = [tok.text for tok in raw_toks]

    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(text)

    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    spans = labeled_by_hand_df['annotations'][labeled_by_hand_index][0]['result']

    for e in spans:
        e = e['value']
        entity_type = "ADR"
        e_words = sorted({idx for idx in char2word[e['start']:e['end']] if idx is not None})
        word_labels[e_words[0]] = 'B-' + entity_type
        for idx in e_words[1:]:
            word_labels[idx] = 'I-' + entity_type

    return {'tokens': words, 'tags': word_labels}

In [ ]:
ner_data_drug = []

for item in drugs:
    dct = extract_labels_drug(item)
    if dct != 0:
        ner_data_drug.append(dct)

len(ner_data_drug)

376

In [ ]:
ner_data_by_hand = [extract_labels_by_hand(i) for i in labeled_by_hand_df.index]

In [ ]:
ner_data = [extract_labels(i) for i in range(0, 30)]

ner_data = ner_data + ner_data_by_hand + ner_data_drug

ner_train, ner_test = train_test_split(ner_data, test_size=0.1, random_state=200)
ner_train = ner_data

In [ ]:
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(3)

,tokens,tags
361,"[В, первый, раз, ,, когда, ребенок, заболел, обычным, орви, ,, нам, сразу, же, врач, навыписывал, целый, ряд, лекарств, ,, в, том, числе, и, антибиотик, "", бактрим, "", ,, обычно, должны, сразу, же, выписать, к, нему, Линекс, ,, для, улучшения, пищеварения, ,, но, никто, нам, ничего, не, прописал...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
158,"[Подскажите, советом, !!!, Теленку, 3, недели, взяли, две, недели, назад, на, ферме, ., Неделю, назад, стала, выпадать, шерсть, местами, клочками, шея, ,, лоб, ,, ноги, у, заднего, прохода, ,, сейчас, еще, и, начал, разлизывать, эти, лысины, ., Кормим, ЗЦМ, в, сутки, 6, литров, ,, комбикорм, нач...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ADR, I-ADR, I-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
480,"[В, общем, сделали, моему, дитёнку, эту, АКДС, ,, кроме, жуткого, плача, (, после, которого, он, проспал, 3, часа, ), никаких, последствий, не, было, .]","[O, O, O, O, O, O, O, O, O, O, B-ADR, O, O, O, O, O, O, O, O, O, O, O, O, O]"


In [ ]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 670
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 67
    })
})

In [ ]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O', 'B-ADR', 'I-ADR']

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/16af2afaa4ceaa8d50b689bd4c2f7ef7fe3bfac06c0aac7d82a5c1c72298b62a.cc3312d07ccf88871a3c2b7cb3442138e6785101efead94d9f77e96301cf7f4a
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/cfa6d82dc8ecc7fe3f06deb449f38968cdd188bb84c8da0e06f0bbfddbede1e3.550ab7157d36210bf96c7c3b30e621933d37d635c5f2e290f7e88bd5f7c9198a
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/20317640533199c6b37a557395cd5ee5fcb8777be7c89bb1314bfd43058b35e9.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolv

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

loading configuration file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/726139048d10597682731a7a4c0b8ef0382927911bc0f6f050a4f7f0afb04c2a.149cdc07694f3925e290abc5528c57a543bcbc9af955c0202b1028584ad15cb4
Model config BertConfig {
  "_name_or_path": "/gd/MyDrive/models/rubert-tiny-mlm-nli-sentence",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0

In [ ]:
# args = TrainingArguments(
#     "ner",
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     save_strategy='no',
#     report_to='none',
# )

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
# example = ner_train[4]
# labels = example['tags']
# metric.compute(predictions=[labels], references=[labels])

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# trainer = Trainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["test"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [ ]:
# trainer.evaluate()

In [ ]:
# for param in model.bert.parameters():
    # param.requires_grad = False

In [ ]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)
#         print(param)

In [ ]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [ ]:
# trainer.train()

In [ ]:
#разморозка
for param in model.parameters():
    param.requires_grad = True

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=16,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

PyTorch: setting up devices


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.272117,0.528814,0.636735,0.577778,0.911941
2,No log,0.148685,0.701149,0.746939,0.723320,0.952096
3,0.267900,0.085270,0.791667,0.853061,0.821218,0.972878
4,0.267900,0.041331,0.920949,0.951020,0.935743,0.990842
5,0.267900,0.023614,0.952756,0.987755,0.969940,0.995069
6,0.066700,0.023106,0.893536,0.959184,0.925197,0.990842
7,0.066700,0.011830,0.971888,0.987755,0.979757,0.997534
8,0.066700,0.008166,0.979920,0.995918,0.987854,0.997887
9,0.024300,0.002702,0.987854,0.995918,0.991870,0.999296
10,0.024300,0.003961,0.983936,1.000000,0.991903,0.998591


TrainOutput(global_step=2688, training_loss=0.06991522666066885, metrics={'train_runtime': 112.6336, 'train_samples_per_second': 95.176, 'train_steps_per_second': 23.865, 'total_flos': 15289883012160.0, 'train_loss': 0.06991522666066885, 'epoch': 16.0})

In [ ]:
trainer.evaluate()

{'epoch': 16.0,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_loss': 0.00026734662242233753,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_runtime': 0.2772,
 'eval_samples_per_second': 241.703,
 'eval_steps_per_second': 61.328}

In [ ]:
# predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
# predictions = np.argmax(predictions, axis=2)

# # Remove ignored index (special tokens)
# true_predictions = [
#     [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]
# true_labels = [
#     [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
#     for prediction, label in zip(predictions, labels)
# ]

# results = metric.compute(predictions=true_predictions, references=true_labels)
# results

In [ ]:
# tokens = tokenizer(text, return_tensors='pt')
# tokens = {k: v.to(model.device) for k, v in tokens.items()}

# with torch.no_grad():
#     pred = model(**tokens)
# pred.logits.shape

In [ ]:
# indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
# token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
# for t, idx in zip(token_text, indices):
#     print(f'{t:15s} {label_list[idx]:10s}')

In [ ]:
model.save_pretrained('ner_bert_top14/')
tokenizer.save_pretrained('ner_bert_top14/')

Configuration saved in ner_bert_top14/config.json
Model weights saved in ner_bert_top14/pytorch_model.bin
tokenizer config file saved in ner_bert_top14/tokenizer_config.json
Special tokens file saved in ner_bert_top14/special_tokens_map.json


('ner_bert_top14/tokenizer_config.json',
 'ner_bert_top14/special_tokens_map.json',
 'ner_bert_top14/vocab.txt',
 'ner_bert_top14/added_tokens.json',
 'ner_bert_top14/tokenizer.json')

In [ ]:
from transformers import pipeline
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)

In [ ]:
# !wget -q -O test.csv https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Wo70d4_PAwujqA

In [ ]:
test_df = pd.read_csv('test.csv')

In [ ]:
d = 0
print(test_df.text[d])
pipe(test_df.text[d])

Понос у месячных телят. Подскажите методы и способы лечения жидкого стула у месячных телят. Хватают в рот и лижут всё что видят, сено, солому, лижут соль. Потом страдают. И мы вместе с ними.


[{'end': 5,
  'entity_group': 'ADR',
  'score': 0.99978495,
  'start': 0,
  'word': 'Понос'}]

In [ ]:
ner_model.hugginface_pipeline(test_df.text[d])

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


[{'end': 86,
  'entity_group': 'ADR',
  'score': 0.7198193,
  'start': 79,
  'word': 'твёрдая'}]

In [ ]:
from ner_scripts2.spans_prediction import NER
ner_model = NER('ner_bert_top12/', 'ner_bert_top12/')

loading configuration file ner_bert_top12/config.json
Model config BertConfig {
  "_name_or_path": "cointegrated/rubert-tiny2",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "O",
    "1": "B-ADR",
    "2": "I-ADR"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "B-ADR": 1,
    "I-ADR": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 83828
}

loading weights file ner_bert_top12/pytorch_model.bin
All model checkpoint weights wer

In [ ]:
test_df['span'] = test_df['text'].apply(lambda x: ner_model.predict_spans(x))

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
test_df.to_json('test_df_with_spans14.json')

In [ ]:
!zip -r /content/ner_bert_top14.zip /content/ner_bert_top14

  adding: content/ner_bert_top14/ (stored 0%)
  adding: content/ner_bert_top14/vocab.txt (deflated 64%)
  adding: content/ner_bert_top14/pytorch_model.bin (deflated 8%)
  adding: content/ner_bert_top14/special_tokens_map.json (deflated 40%)
  adding: content/ner_bert_top14/tokenizer.json (deflated 65%)
  adding: content/ner_bert_top14/tokenizer_config.json (deflated 42%)
  adding: content/ner_bert_top14/config.json (deflated 49%)


In [ ]:
from google.colab import files
files.download("/content/ner_bert_top14.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>